In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [3]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
boiling_plan_df.head()

,id,boiling,sku,kg
0,1.0,<Boiling 2>,<SKU 1>,850
1,2.0,<Boiling 1>,<SKU 34>,850
2,3.0,<Boiling 1>,<SKU 4>,567
3,3.0,<Boiling 1>,<SKU 14>,283
4,4.0,<Boiling 1>,<SKU 4>,850


In [3]:
cheese_maker_lines = {str(i): Block('cheese_maker', i=i) for i in range(4)}
cheese_maker_lines

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [96]:
b = root.children[0]
b['pouring'].props.get_all_props()

{'class': 'pouring',
 'y': 0,
 't': 48,
 'pouring_line': None,
 'time_size': 135,
 'i': 0,
 'size': 27}

In [ ]:
# todo: inherit before cutting properly

In [111]:
boilings = [node for node in root.children if node.props['class'] == 'boiling']
for boiling in boilings:
    add_push(cheese_maker_lines[boiling.props['pouring_line']], boiling['pouring'], new_props={boiling['pouring'].props.get_all_props()})

In [112]:
cheese_maker_lines['0']

TypeError: unsupported operand type(s) for +: 'int' and 'dict'

In [104]:
cheese_maker_lines['3']

cheese_maker (0, 0]
  pouring (39, 64]
    block (39, 39]
      termizator (39, 45]
      pouring_name (45, 64]
    block (39, 39]
      pouring_and_fermenting (39, 49]
      soldification (49, 55]
      cutting (55, 59]
      pouring_off (59, 62]
      extra (62, 64]
  pouring (122, 147]
    block (122, 122]
      termizator (122, 128]
      pouring_name (128, 147]
    block (122, 122]
      pouring_and_fermenting (122, 132]
      soldification (132, 138]
      cutting (138, 142]
      pouring_off (142, 145]
      extra (145, 147]
  pouring (148, 173]
    block (148, 148]
      termizator (148, 154]
      pouring_name (154, 173]
    block (148, 148]
      pouring_and_fermenting (148, 158]
      soldification (158, 164]
      cutting (164, 168]
      pouring_off (168, 171]
      extra (171, 173]

In [ ]:
row_style_prod = {
    'pouring': {'y': lambda b: {'0': 6, '1': 9, '2': 15, '3': 18}[b.props['pouring_line']]},
    'melting': {'y': lambda b: 24 if b.props['boiling_type'] == 'water' else {'0': 33, '1': 36, '2': 39, '3': 42}[b.props['melting_line']]},
    'packing_and_preconfiguration': {'y': lambda b: 29 if b.props['boiling_type'] == 'water' else 46},
    'cleaning': {'y': 12}
}

row_style = row_style_dev if mode == 'dev' else row_style_prod


In [12]:
root = make_schedule(boiling_plan_df)
# wb = draw_workbook(root, mode='prod')
# wb.save('output.xlsx')
# open_file_in_os('output.xlsx') 

In [62]:
pyperclip.copy(draw_print(root, visible_only=True))

In [ ]:
style = load_style(mode='prod')
root.props.update({'size': max(c.end for c in root.children)})
init_sheet_func = init_sheet if mode == 'dev' else init_template_sheet(template_fn=template_fn)
draw_schedule(root, style, init_sheet_func=init_sheet_func)

In [115]:
cleanings = Block('cleaning')


In [117]:
cleanings

cleaning (0, 0]
  cleaning (94, 110]
    full_cleaning (94, 110]
  cleaning (174, 182]
    short_cleaning (174, 182]
  cleaning (199, 215]
    full_cleaning (199, 215]

# Drawing

In [353]:
def add_push_with_props(parent, block):
    return add_push(parent, block, new_props=block.props.get_all_props())

mode = 'prod'
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

root = make_schedule(boiling_plan_df)

with make('schedule'):
    # todo: make properly
    make(orient='vertical', h=1, visible=False)
    with make('header', orient='vertical', h=1, index_width=3):
        make(h=1, size=1, text='График наливов', push_func=add_push)
        for i in range(288):
            cur_time = cast_time(i + cast_t('01:00'))
            if cur_time[-2:] == '00':
                make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    for i in range(2):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)

    with make('cleanings', orient='vertical', h=2):
        for block in root['cleaning']:
            make(block, push_func=add_push_with_props)
    make(orient='vertical', h=1, visible=False)

    for i in range(2, 4):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)
    
    with make('header', orient='vertical', h=1, index_width=4):
        for i in range(288):
            cur_time = cast_time(i + cast_t('07:00'))
            if cur_time[-2:] == '00':
                make(t=i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    with make('water_melting', orient='vertical', h=4):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['melting'], push_func=add_push_with_props)
                
    with make('water_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
    
    for i in range(5):
        # todo: make properly, hardcode
        if any(block.props['melting_line'] == str(i) for block in root['boiling']):
            with make(f'salt_melting_{i}', orient='vertical', h=3):
                for block in root['boiling']:
                    if block.props['boiling_type'] == 'salt' and block.props['melting_line'] == str(i):
                        make(block['melting_and_packing']['melting'], push_func=add_push_with_props)
    
    with make('water_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'salt':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
row_root = maker.root

In [354]:
row_root.props.update({'h': max(c.end for c in root.children)})
style = load_style()
init_sheet_func = init_sheet if mode == mode else init_template_sheet(template_fn=template_fn)
wb = draw_schedule(row_root, style, init_sheet_func=init_sheet_func)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 

block 4 1 1 1
block 5 1 1 1
block 6 1 1 1
block 7 1 1 1
block 8 1 1 1
block 9 1 1 1
block 10 1 1 1
block 11 1 1 1
block 12 1 1 1
block 13 1 1 1
block 14 1 1 1
block 15 1 1 1
block 16 1 1 1
block 17 1 1 1
block 18 1 1 1
block 19 1 1 1
block 20 1 1 1
block 21 1 1 1
block 22 1 1 1
block 23 1 1 1
block 24 1 1 1
block 25 1 1 1
block 26 1 1 1
block 27 1 1 1
block 28 1 1 1
block 29 1 1 1
block 30 1 1 1
block 31 1 1 1
block 32 1 1 1
block 33 1 1 1
block 34 1 1 1
block 35 1 1 1
block 36 1 1 1
block 37 1 1 1
block 38 1 1 1
block 39 1 1 1
block 40 1 1 1
block 41 1 1 1
block 42 1 1 1
block 43 1 1 1
block 44 1 1 1
block 45 1 1 1
block 46 1 1 1
block 47 1 1 1
block 48 1 1 1
block 49 1 1 1
block 50 1 1 1
block 51 1 1 1
block 52 1 1 1
block 53 1 1 1
block 54 1 1 1
block 55 1 1 1
block 56 1 1 1
block 57 1 1 1
block 58 1 1 1
block 59 1 1 1
block 60 1 1 1
block 61 1 1 1
block 62 1 1 1
block 63 1 1 1
block 64 1 1 1
block 65 1 1 1
block 66 1 1 1
block 67 1 1 1
block 68 1 1 1
block 69 1 1 1
block 70 1 1 1
b

block 216 1 17 1
block 217 1 17 1
block 218 1 17 1
block 219 1 17 1
block 220 1 17 1
block 221 1 17 1
block 222 1 17 1
block 223 1 17 1
block 224 1 17 1
block 225 1 17 1
block 226 1 17 1
block 227 1 17 1
block 228 1 17 1
block 229 1 17 1
block 230 1 17 1
block 231 1 17 1
block 232 1 17 1
block 233 1 17 1
block 234 1 17 1
block 235 1 17 1
block 236 1 17 1
block 237 1 17 1
block 238 1 17 1
block 239 1 17 1
block 240 1 17 1
block 241 1 17 1
block 242 1 17 1
block 243 1 17 1
block 244 1 17 1
block 245 1 17 1
block 246 1 17 1
block 247 1 17 1
block 248 1 17 1
block 249 1 17 1
block 250 1 17 1
block 251 1 17 1
block 252 1 17 1
block 253 1 17 1
block 254 1 17 1
block 255 1 17 1
block 256 1 17 1
block 257 1 17 1
block 258 1 17 1
block 259 1 17 1
block 260 1 17 1
block 261 1 17 1
block 262 1 17 1
block 263 1 17 1
block 264 1 17 1
block 265 1 17 1
block 266 1 17 1
block 267 1 17 1
block 268 1 17 1
block 269 1 17 1
block 270 1 17 1
block 271 1 17 1
block 272 1 17 1
block 273 1 17 1
block 274 1 17

In [330]:
root['boiling'][0]

boiling (48, 147]
  pouring (48, 75]
    block (48, 48]
      termizator (48, 54]
      pouring_name (54, 75]
    block (48, 48]
      pouring_and_fermenting (48, 56]
      soldification (56, 63]
      cutting (63, 70]
      pouring_off (70, 73]
      extra (73, 75]
  drenator (75, 118]
  melting_and_packing (118, 147]
    melting (118, 138]
      block (118, 118]
        serving (118, 124]
      block (118, 118]
        serving (118, 124]
        melting_label (124, 128]
        melting_name (128, 138]
      block (118, 118]
        serving (118, 124]
        melting_process (124, 138]
      block (118, 118]
        block (118, 124]
        cooling1 (124, 128]
        cooling2 (128, 132]
    packing_and_preconfiguration (132, 147]
      packing (132, 147]
        block (132, 132]
          packing_label (132, 135]
          packing_name (135, 147]
        block (132, 132]
          packing_brand (132, 147]
        block (132, 132]
          configuration (132, 133]
          configura

In [314]:
row_root['schedule']['water_melting'].props['y']

17

In [319]:
row_root['schedule']['water_melting'][0].props.relative_props['y']

10

In [275]:
row_root['schedule']['cheese_maker_0']['pouring'][0].props.get_all_props()

{'y': 2,
 'boiling_label': '3.6 Альче  8000кг',
 'boiling_id': 9,
 'time_size': 135,
 'index_width': 4,
 'pouring_line': '0',
 'block_num': 1,
 't': 48,
 'h': 1,
 'class': 'pouring',
 'boiling_type': 'water',
 'beg_time': '01:00',
 'size': 27}

In [277]:
cast_time(48 - 12 + 5)

'03:25'